In [ ]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

title_list = []
date_list = []
text_list = []
link_list = []

cnt = 0 
omission_list_2 = []

for i in tqdm(url_list[626:1001]): #323
    cnt += 1 
    
    url = urllib.request.Request(i,headers=headers) #헤더 추가 http 오류 해결
    
    try: #Connection reset by peer -exception
        res = urllib.request.urlopen(url).read() 
        soup = BeautifulSoup(res,  "html.parser", from_encoding='utf-8')


        title = soup.find_all('h1', {'class': 'headline'}) if soup.find_all('p', {'class': 'headline'}) is not None else soup.find_all('h1', {'class': 'article_head'})
        date = soup.select('em')[0].get_text()[5:15]

         #본문 
        content_of_article = soup.select('div.art_body') 
        for item in content_of_article: 
            string_item = str(item.find_all("p", {"class": "content_text"})) 
            text_list.append(string_item)  

        try:    
            title_list.append(title[0].text) #IndexError 예외처리
            date_list.append(date)
            link_list.append(i)

        except IndexError:
            omission_list_2.append(i)
            
    except SocketError as e:
        if e.errno != errno.ECONNRESET:
            raise # Not error we are looking for
    pass # Handle error here.


In [1]:
#cell 크기 조절
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 85% !important; }</style>"))

#display all text in a cell without truncation
import pandas as pd
print("pandas version: ", pd.__version__)
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

pandas version:  1.1.3


In [3]:
 # -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
from datetime import datetime
import requests 

from tqdm import tqdm
from time import sleep
import pandas as pd
import time
import urllib.request 
from bs4 import BeautifulSoup
from random import randint

from socket import error as SocketError
import errno

# 링크/섹션 가져오기

In [109]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

target = ['문화','정치','사회','경제']

section_list=[]
url_list = []

for page in tqdm(range(366,2971)): #365-2970
    url = f'https://search.khan.co.kr/search.html?stb=khan&dm=0&q=%EC%B2%AD%EB%85%84&pg={page}&sort=1'
    req = requests.get(url,headers = headers)#headers = headers
    soup = BeautifulSoup(req.content, 'html.parser')
    

    for contents_type in soup.select('dd.date > strong'):
        if contents_type.get_text() in target:
            section_list.append(contents_type.get_text()) #섹션 추가
            link1 = contents_type.parent.parent
            link2 = link1.select_one('dt > a')
            url_list.append(link2['href'])

            
# for u in url_list :
#     print(u, end = '\n')

100%|██████████| 2166/2166 [05:37<00:00,  6.42it/s]


In [14]:
print(len(url_list))
print(len(section_list))

13987
13987


In [117]:
#for page in tqdm(range(365,2531)): #365-2968
print(len(url_list))
print(len(section_list))

12120
12120


In [15]:
# df = pd.DataFrame({
#                   '섹션':section_list,
#                   'url':url_list})

# df.to_csv('/Users/dawon/Desktop/학준님숙제(신문사크롤링)/경향_링크섹션.csv', index = False, encoding = 'utf-8-sig')

# 각종 정보 크롤링

In [199]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

title_list = []
date_list = []
text_list = []
link_list = []
section_list=[]
url_list = []

target = ['문화','정치','사회','경제']

for page in tqdm(range(355,368)): #366-2970 to 2530 /1685,2201
    
    url = f'https://search.khan.co.kr/search.html?stb=khan&dm=0&q=%EC%B2%AD%EB%85%84&pg={page}&sort=1'
    req = requests.get(url,headers = headers)#headers = headers
    soup = BeautifulSoup(req.content, 'html.parser')

    #링크 가져오기 
    for contents_type in soup.select('dd.date > strong'):
        if contents_type.get_text() in target:
            section_list.append(contents_type.get_text()) #섹션 추가
            link1 = contents_type.parent.parent
            link2 = link1.select_one('dt > a')
            url_list.append(link2['href']) #타겟이 되는 섹션의 링크만 추가

            
            url = urllib.request.Request(link2['href'],headers=headers)
            try: #Connection reset by peer -exception
                res = urllib.request.urlopen(url).read() 
                soup = BeautifulSoup(res,  "html.parser", from_encoding='utf-8')

                #제목
                title = soup.find_all('h1', {'class': 'headline'}) if soup.find_all('p', {'class': 'headline'}) is not None else soup.find_all('h1', {'class': 'article_head'})
                
                #날짜
                date = soup.select('em')[0].get_text()[5:15]

                 #본문 
                text_cont = soup.find('div', {'class' : 'art_body'}).text.replace('\n', "")
                text_list.append(text_cont)
                
            
                try:    
                    title_list.append(title[0].text) #IndexError 예외처리
                    date_list.append(date)
                    link_list.append(url)

                except IndexError:      
                    date_list.append('예외')
                    title_list.append('예외')
                    text_list.append('예외')
                    
            except SocketError as e:
                date_list.append('예외')
                title_list.append('예외')
                text_list.append('예외')
                if e.errno != errno.ECONNRESET:
                    raise # Not error we are looking for
            pass # Handle error here.


  0%|          | 0/13 [00:00<?, ?it/s]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
  8%|▊         | 1/13 [00:05<01:01,  5.15s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

In [200]:
print(len(link_list))
print(len(section_list))
print(len(text_list))
print(len(date_list))
print(len(title_list))

88
88
88
88
88


In [201]:
df = pd.DataFrame({'날짜':date_list,
                  '언론사명':'경향신문',
                  '섹션':section_list,
                  '제목':title_list,
                  '내용':text_list,
                  'url':url_list})

df['내용'] = df['내용'].apply(lambda x : x.replace('\n',''))

df.to_csv('/Users/dawon/Desktop/학준님숙제(신문사크롤링)/경향_20년도추가.csv', index = False, encoding = 'utf-8-sig')

In [449]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

title_list = []
date_list = []
text_list = []
link_list = []

cnt = 0 
omission_list_2 = []

for url in tqdm(url_list): 
    cnt += 1 
    
    try : 
        # 기사 정보 초기화
        title = None
        date =  None
        section = None
        text = None 
        
    
        url = urllib.request.Request(i,headers=headers) #헤더 추가 http 오류 해결
        res = urllib.request.urlopen(url).read() 
        if req.status_code == 200 : 
            soup = BeautifulSoup(res,  "html.parser", from_encoding='utf-8')

            title = soup.select('h1.headline')
            date = soup.select('em')[0].get_text()[5:15]
             #본문 
            content_of_article = soup.select('div.art_body') 
            for item in content_of_article: 
                string_item = str(item.find_all("p", {"class": "content_text"})) 
                text_list.append(string_item)  

            # 리스트에 입력
            title_list.append(title[0].text)
            date_list.append(date)
            link_list.append(i)

        else : 
            omission_list_2.append(url)
    except Exception as e :
        omission_list_2.append(url)

100%|██████████| 7/7 [00:04<00:00,  1.57it/s]


In [276]:
text_list

['경기회복에다 기저효과가 더해지면서 6월 취업자수가 60만명 가까이 늘었다. 양적으로는 4개월 연속 증가세다. 하지만 취업자 증가폭은 두 달째 감소한데다, 구직단념자도 전년보다 4만6000명이 늘었다. 주18시간 미만 일하는 초단시간근로자도 급증했다. 코로나19 4차 대유행 여파로 7월 지표는 더 악화될 것으로 우려된다.',
 '홍남기 부총리 겸 기획재정부 장관은 “고용의 양적 측면뿐 아니라 세부 내용 측면에서도 개선세가 점차 뚜렷해지고 있다”고 14일 밝혔다.',
 '중소기업에 다니는 청년들을 대상으로 한 저금리 전세자금 대출의 운영기간이 내년 2023년까지 연장된다. 청년 대상 월세대출은 자격요건이 완화되고, 월세 대출한도도 월 50만원으로 상향된다.',
 '문재인 대통령이 ‘한국판 뉴딜 종합계획’ 발표 1주년을 맞은 14일 한국판 뉴딜 총투자 규모를 2025년까지 기존 160조원에서 220조원으로 60조원 확대하겠다고 밝혔다.',
 '정부가 ‘한국판 뉴딜 2.0’을 14일 발표했다. 지난해 발표한 한국판 뉴딜 1.0에다 코로나19 사태 이후 심각해진 청년·양극화 대책을 추가하고, 디지털화와 탄소중립 추진 방안을 보강한 것이다. 정부는 2025년까지 기존보다 60조원 늘어난 220조원을 투입해 일자리 250만개를 만들겠다는 구상이지만, ‘청년 용돈정책’ 논란이 불거지는데다 차기 정권에서 정책이 유지될지 미지수라는 회의론이 나온다.',
 '세계 최초 시각장애인 공인재무분석사(CFA)이자 미국 월가의 애널리스트. 신순규씨(53·사진) 앞에 붙는 수식어다. 그의 스펙은 특이하고, 일견 대단하다. 신씨는 피아노를 배우기 위해 15살에 홀로 미국 유학을 떠났다가 진로를 바꿔 하버드와 MIT를 졸업한 후 투자은행 JP모건에 입사했다. 현재는 브라운 브라더스 해리먼에서 일하는 27년차 베테랑 채권 애널리스트다. 동시에 그는 따뜻한 마음을 가진 에세이스트이기도 하다. 2015년 일상에서의 소소한 일화와 긍정의 메시지를 담은 <눈 감으면 보이는 것들>에 이어 최근 두 번

In [5]:
def crawler():
#     list_url = url_list 
    f = open(RESULT_PATH+'ydata3.txt', 'w', encoding='UTF-8')
    
    for i in tqdm(url_list):
        url = urllib.request.Request(i)
        res = urllib.request.urlopen(url).read().decode("utf-8")  
        soup = BeautifulSoup(res,  "html.parser", from_encoding='utf-8')


        # 날짜 가져오기
        pdate = soup.select('div > em:nth-child(1)').get_text()[5:15]
        f.write(pdate)

        # 언론명 가져오기
        newss = '경향신문'
        f.write(newss)

        #섹션
        section = soup.select('li > a')[1].get_text()[5:15]
        f.write(section)

        # 제목
        title = soup.select('div.subject')  #대괄호는  h3#articleTitle 인 것중 첫번째 그룹만 가져오겠다.
        for item in title: 
            string_item = str(item.find_all(text=True)) 
            f.write(string_item)


        #본문
        content_of_article = soup.select('div.art_body').replace('\n', " ")
        for item in content_of_article: 
            string_item = str(item.find_all("p", {"class": "content_text"})) 
            f.write(string_item.strip())


        #링크
        f.write(i)


    f.close()

            
def excel_make():
    data = pd.read_csv(RESULT_PATH+'ydata3.txt', sep='\t',header=None, error_bad_lines=False)
    data.columns = ['date','news','section','title','contents','link'] 
    #print(data)
    xlsx_outputFileName = '%s-%s-%s %s시 %s분 %s초 result.xlsx' % (now.year, now.month, now.day, now.hour, now.minute, now.second)
    xlsx_name = 'result' + '.xlsx'
    data.to_excel(RESULT_PATH+xlsx_outputFileName, encoding='utf-8')


def main():
    crawler() 
    excel_make() 
    
main()

  0%|          | 0/7 [00:00<?, ?it/s]


HTTPError: HTTP Error 403: Forbidden

In [162]:
""" 경향신문 특정 키워드를 포함하는, 특정 날짜 이전 기사 내용 크롤러(정확도순 검색) 
python [모듈 이름] [키워드] [가져올 페이지 숫자] [결과 파일명] 한 페이지에 기사 10개 """ 
import sys 
from bs4 import BeautifulSoup 
import urllib.request 
from urllib.parse import quote 
from urllib.request import Request, urlopen 

TARGET_URL_BEFORE_PAGE_NUM = "http://search.khan.co.kr/search.html?stb=khan" 
TARGET_URL_BEFORE_KEWORD = '&q=' 
TARGET_URL_PAGE = '&pg=' 
TARGET_URL_SORT = '&sort=2' 

# 기사 검색 페이지에서 기사 제목에 링크된 기사 본문 주소 받아오기 d
def get_link_from_news_title(page_num, URL, output_file): 
    for i in range(page_num): 
        print(i) 
        current_page_num = i + 1 
        URL_with_page_num = URL + TARGET_URL_PAGE + str(current_page_num) + TARGET_URL_SORT 
        source_code_from_URL = urllib.request.urlopen(URL_with_page_num) 
        soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8') 
        
        for title in soup.find_all('dl', 'phArtc'): 
            title_link = title.select('a') 
            article_URL = title_link[0]['href'] 
            get_text(article_URL, output_file) 
            
        # 기사 본문 내용 긁어오기 (위 함수 내부에서 기사 본문 주소 받아 사용되는 함수) 
        
def get_text(URL, output_file): 
    req = Request(URL, headers={'User-Agent': 'Mozilla/5.0'}) 
    source_code_from_url = urlopen(req).read() 
    soup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding='utf-8') 
    
    # 제목 
    content_of_article = soup.select('div.subject') 
    for item in content_of_article:
        string_item = str(item.find_all(text=True)) 
        output_file.write(string_item) 
        
    # 본문 
    content_of_article = soup.select('div.art_body') 
    for item in content_of_article: 
        string_item = str(item.find_all("p", {"class": "content_text"})) 
        output_file.write(string_item) 
        
# 메인함수 

def main(argv): 
    
    if len(argv) != 4: 
        print("python [모듈이름] [키워드] [가져올 페이지 숫자] [결과 파일명]") 
        return 
    
    keyword = argv[1] 
    page_num = int(argv[2]) 
    output_file_name = argv[3] 
    target_URL = TARGET_URL_BEFORE_PAGE_NUM + TARGET_URL_BEFORE_KEWORD + quote(keyword)
    output_file = open(output_file_name, 'w', encoding='UTF-8') 
    get_link_from_news_title(page_num, target_URL, output_file) 
    output_file.close() 

if __name__ == '__main__': 
    main(sys.argv)



0
1


In [161]:
sys.argv = "python3 청년 2 test.txt".split()

In [115]:
def get_link_from_news_title(page_num, URL, output_file): 
    for i in range(page_num): 
        current_page_num = i + 1 
        URL_with_page_num = URL + TARGET_URL_PAGE + str(current_page_num) + TARGET_URL_SORT 
        source_code_from_URL = urllib.request.urlopen(URL_with_page_num) 
        soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8') 
        
        for title in soup.find_all('dl', 'phArtc'): 
            title_link = title.select('a') 
            article_URL = title_link[0]['href'] 
            get_text(article_URL, output_file)



In [57]:
   
#     for urls in soup.select("._sp_each_url"):
#         try :
#             #print(urls["href"])
#             if urls["href"].startswith("https://news.naver.com"):
#                 #print(urls["href"])
#                 news_detail = get_news(urls["href"])
#                 # pdate, pcompany, title, btext
#                 f.write("{}\t{}\t{}\t{}\t{}\n".format(news_detail[1], news_detail[4], news_detail[0], news_detail[2],news_detail[3])) # new style
#         except Exception as e:
#             print(e)
#             continue

# soup.find_all('div', class_=lambda x: x not in classToIgnore

#     contents_link = soup.select('dl.phArtc > dt.a')
# #     print(contents_link)
#     for content in contents_link:
#         print(content.get_text())

#     for contents_type in soup.select('dd.date > strong'):
#         print(contents_type.get_text())
    
    
#                if contents_type.get_text() in target:
#                     url = content.find('a').get('href')
#                     url_list.append(url)

        
# for u in url_list :
#     print(u, end = '\n')

In [8]:
url_list = []
for page in tqdm(range(2)) : 
    url = f'https://news.joins.com/Search/TotalNews?page={page}&Keyword=%EC%B2%AD%EB%85%84&SortType=New&SearchCategoryType=TotalNews'
    req = requests.get(url) #headers = headers
    soup = BeautifulSoup(req.content, 'html.parser')

    contents = soup.select('ul.list_default > li div.text') # > 문자 앞뒤로 공백이 꼭 필요함 
    for content in contents :
        url = content.find('a').get('href')
        url_list.append(url)
        
for u in url_list :
    print(u, end = '\n')

  0%|          | 0/2 [00:00<?, ?it/s]


NameError: name 'requests' is not defined